<a href="https://colab.research.google.com/github/blackhawkee/llm-play/blob/main/stable_diffusion_xl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U autotrain-advanced
!autotrain setup --update-torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 27.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 MB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 38.1 MB/s eta

2023-07-27 19:25:38.928502: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
> INFO    Installing latest transformers@main
> INFO    Successfully installed latest transformers
> INFO    Installing latest peft@main
> INFO    Successfully installed latest peft
> INFO    Installing latest diffusers@main
> INFO    Successfully installed latest diffusers
> INFO    Installing latest trl@main
> INFO    Successfully installed latest trl
> INFO    Ins

In [2]:
import requests
import os
images = ["https://c4.wallpaperflare.com/wallpaper/158/382/512/5bd34deb802eb-wallpaper-preview.jpg",
            "https://c1.wallpaperflare.com/preview/51/466/512/dog-mascot-labrador-animal.jpg",
            "https://cdn.pixabay.com/photo/2016/02/19/15/46/labrador-retriever-1210559_1280.jpg",
            "https://c1.wallpaperflare.com/preview/512/113/145/dog-cute-pet-canidae.jpg"
            ]

images_directory = "/content/images"
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

for idx, url in enumerate(images):
  response = requests.get(url)
  with open(f"{images_directory}/image_{str(idx)}.jpg", "wb") as f:
      f.write(response.content)

In [3]:
!autotrain dreambooth \
--model stabilityai/stable-diffusion-xl-base-1.0 \
--output output/ \
--image-path /content/images/ \
--prompt "photo of sks dog" \
--resolution 512 \
--batch-size 1 \
--num-steps 500 \
--fp16 \
--gradient-accumulation 4 \
--lr 1e-4 \
--use-8bit-adam

2023-07-27 19:29:39.219804: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
> INFO    Namespace(version=False, model='stabilityai/stable-diffusion-xl-base-1.0', revision=None, tokenizer=None, image_path='/content/images/', class_image_path=None, prompt='photo of sks dog', class_prompt=None, num_class_images=100, class_labels_conditioning=None, prior_preservation=None, prior_loss_weight=1.0, output='output/', seed=42, resolution=512, center_crop=None, train_text_encoder=None, batch_size=1, sample_batch_size=4, epochs=1, num_steps=500, checkpointing_steps=500, resume_from_checkpoint=None, gradient_accumulation=4, gradient_checkpointing=None, lr=0.0001, scale_lr=None, scheduler='constant', warmup_steps=0, num_cycles=1, lr_power=1.0, dataloader_num_workers=0, use_8bit_adam=True, adam_beta1=0.9, adam_beta2=0.999, adam_weight_decay=0.01, adam_epsilon=1e-08, max_grad_norm=1.0, allow_tf32=None, prior_generation_precision=None, local_rank=-1, xfor

In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch

model = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(
    model,
    torch_dtype = torch.float16
)
pipe.to("cuda")
pipe.load_lora_weights(
    "model/", weight_name = "pytorch_lora_weights.safetensors"
)

refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    torch_dtype = torch.float16
)
refiner.to("cuda")

prompt = "portrait of sks dog, cartoon, pixar, 3d, headshots, fantasy, 4k, uhd"
for seed in range(30):
  generator = torch.Generator("cuda").manual_seed(seed)
  image = pipe(
      prompt=prompt, generator=generator, num_inference_steps=25
  )
  image = image.images[0]
  image = refiner(prompt=prompt, generator=generator, image=image)
  image=image.images[0]
  image.save(f"images/{seed}.png")